In [1]:
import torch
from math import factorial, ceil
# from pytorch3d.io import load_obj
import timeit
import numpy as np

In [2]:
v_matrix = [[0, 0, 0], [0, 0, 1], [0, 1 ,0], [0, 1, 1], [1, 0, 0], [1, 0, 1], [1, 1 ,0], [1, 1 ,1]]
f_matrix = [[0,2,4], [4,2,6],[0,1,3], [0,3,2], [0,5,1], [0,4,5], [1,7,3], [1,5,7], [4,7,5], [4,6,7], [7,6,2],[7,2,3]]
v_matrix, f_matrix = torch.tensor(v_matrix,dtype=torch.float32), torch.tensor(f_matrix,dtype=torch.long)
max_m = 15
device = 'cpu'

In [3]:
# This can be taken outside and made available as a global constant
f_ijk_s_p = np.zeros((max_m,max_m,max_m))
f_ijk_p_s_2 = np.zeros((max_m,max_m,max_m))
f_ijk_p_s_3 = np.zeros((max_m,max_m,max_m))
for i in range(max_m):
            for j in range(max_m):
                for k in range(max_m):
                    f_ijk_s_p[i,j,k] = (factorial(i+j+k)/(factorial(i)*factorial(j)*factorial(k)))
                    f_ijk_p_s_3[i,j,k] = ((factorial(i)*factorial(j)*factorial(k))/factorial(i+j+k+3))
                    f_ijk_p_s_2[i,j,k] = 1/f_ijk_s_p[i,j,k]/(i+j+k+1)/(i+j+k+2)
                    f_ijk_p_s_3[i,j,k] = f_ijk_p_s_2[i,j,k]/(i+j+k+3)
f_ijk_s_p = torch.tensor(f_ijk_s_p).float()
f_ijk_p_s_2 = torch.tensor(f_ijk_p_s_2).float()
f_ijk_p_s_3 = torch.tensor(f_ijk_p_s_3).float()

In [4]:
def M_ijk_torch_v6(f_matrix, v_matrix, max_m, is_watertigh, device):
    total_num_faces = len(f_matrix)
    
    loop_batch_size = min(1000, total_num_faces)
    
#     m_v = torch.zeros([max_m,max_m,max_m],dtype=torch.float32, requires_grad=True).to(device)
#     m_s = torch.zeros([max_m,max_m,max_m],dtype=torch.float32, requires_grad=True).to(device)
    m_v = [torch.zeros([1,max_m,max_m],dtype=torch.float32, requires_grad=True).to('cpu') for _i in range(max_m)]
    m_s = [torch.zeros([1,max_m,max_m],dtype=torch.float32, requires_grad=True).to('cpu') for _i in range(max_m)]
    
    
#     # This can be taken outside and made available as a global constant
#     f_ijk_s_p = np.zeros((max_m,max_m,max_m))
#     f_ijk_p_s_2 = np.zeros((max_m,max_m,max_m))
#     f_ijk_p_s_3 = np.zeros((max_m,max_m,max_m))
#     for i in range(max_m):
#                 for j in range(max_m):
#                     for k in range(max_m):
#                         f_ijk_s_p[i,j,k] = (factorial(i+j+k)/(factorial(i)*factorial(j)*factorial(k)))
#                         f_ijk_p_s_3[i,j,k] = ((factorial(i)*factorial(j)*factorial(k))/factorial(i+j+k+3))
#                         f_ijk_p_s_2[i,j,k] = 1/f_ijk_s_p[i,j,k]/(i+j+k+1)/(i+j+k+2)
#                         f_ijk_p_s_3[i,j,k] = f_ijk_p_s_2[i,j,k]/(i+j+k+3)
#     f_ijk_s_p = torch.tensor(f_ijk_s_p)
#     f_ijk_p_s_2 = torch.tensor(f_ijk_p_s_2)
#     f_ijk_p_s_3 = torch.tensor(f_ijk_p_s_3)
    
    
    
    
    for batch_num in range(ceil(total_num_faces/loop_batch_size)):
        start = batch_num*loop_batch_size
        end = min([(batch_num+1)*loop_batch_size, total_num_faces])
        num_faces = end-start
        
        # Extract coordinates
        ABC = v_matrix[f_matrix[start:end,:]]
        if is_watertigh:
            dets = torch.det(ABC).to(device)
        cross = torch.norm(torch.cross(ABC[:,0]-ABC[:,2],ABC[:,1]-ABC[:,2]),p=2,dim = 1).to(device)


        # Allocate Tensors
        # NEW: i dimension of size 2 for M_V, M_S, D,S
        M_V_tensor = torch.zeros([num_faces,max_m,max_m,max_m],dtype=torch.float32).to(device)
        M_S_tensor = torch.zeros([num_faces,max_m,max_m,max_m],dtype=torch.float32).to(device)
        D_tensor = torch.zeros([num_faces,max_m,max_m,max_m],dtype=torch.float32).to(device)
        S_tensor = torch.zeros([num_faces,max_m,max_m,max_m],dtype=torch.float32).to(device)
        # NEW: [:,1,0,0] = 1 instead of [:,0,0,0]
        D_tensor[:,0,0,0] = 1
        S_tensor[:,0,0,0] = 1
        

        jl = torch.arange(max_m)
        kl = torch.arange(max_m)
        # Calculate C,D,S,M Tensors, parallellized over faces
        for i in range(max_m):
            il = torch.tensor(i)
            C_tensor = torch.multiply(torch.multiply(torch.pow(ABC[:,2][:,0].view(-1,1,1,1),il.view(1,-1,1,1)),torch.pow(ABC[:,2][:,1].view(-1,1,1,1),jl.view(1,1,-1,1))),torch.pow(ABC[:,2][:,2].view(-1,1,1,1),kl.view(1,1,1,-1)))*f_ijk_s_p[il.view(-1,1,1),jl.view(1,-1,1),kl.view(1,1,-1)]
            for j in range(max_m):
                for k in range(max_m):
                    # Can get rid of this condition later on if needed
#                     if (i+j+k)<=max_m:
                    # NEW:
                    # In D,S,M_V,M_S:
                    #            i:1
                    #          i-1:0
                    # In C:
                    #            i:0  DONE
                    D_tensor[:,i,j,k] = ABC[:,1][:,0]*D_tensor[:,i-1,j,k]+ABC[:,1][:,1]*D_tensor[:,i,j-1,k]+ABC[:,1][:,2]*D_tensor[:,i,j,k-1]+C_tensor[:,0,j,k]
                    S_tensor[:,i,j,k] = ABC[:,0][:,0]*S_tensor[:,i-1,j,k]+ABC[:,0][:,1]*S_tensor[:,i,j-1,k]+ABC[:,0][:,2]*S_tensor[:,i,j,k-1]+D_tensor[:,i,j,k]

#                         # Absorb this into the next line, to avoid making M_V and M_S tensors
#                         M_V_tensor[:,i,j,k] = f_ijk_p_s_3[i,j,k]*(dets[:]*S_tensor[:,i,j,k])
#                         M_S_tensor[:,i,j,k] = f_ijk_p_s_2[i,j,k]*(cross[:]*S_tensor[:,i,j,k])
                    # Absorb this into the next line, to avoid making M_V and M_S tensors
                    if is_watertigh:
                        M_V_tensor[:,i,j,k] = f_ijk_p_s_3[i,j,k]*(dets[:]*S_tensor[:,i,j,k])
                    M_S_tensor[:,i,j,k] = f_ijk_p_s_2[i,j,k]*(cross[:]*S_tensor[:,i,j,k])

#                         m_v = m_v + torch.sum(M_V_tensor,0)
#                         m_v[i,j,k] = m_v.clone()[i,j,k] + torch.sum(M_V_tensor.clone()[:,i,j,k],0).clone()
#                         m_s = m_s + torch.sum(M_S_tensor,0)

                    #NEW: Instead of summing along 0th axis for each i,j,k, sum along 0th axis for each i
                    #    This requires that we create max_m dijointed tensors, one for each i
            if is_watertigh:
                m_v[i] = m_v[i] + torch.sum(M_V_tensor[:,i,:,:],0)
            m_s[i] = m_s[i] + torch.sum(M_S_tensor[:,i,:,:],0)
            #NEW: Flip D,S,M_V,M_S along i axis
            

 
    m_v = torch.stack(m_v).squeeze()
    m_s = torch.stack(m_s).squeeze()
    
#         m_v = m_v + torch.sum(M_V_tensor,0)
#         m_s = m_s + torch.sum(M_S_tensor,0)
    
    return (m_v,m_s)

In [5]:
def M_ijk_Pozo_v0(f_matrix, v_matrix, max_m, is_watertigh, device):
    total_num_faces = len(f_matrix)
    
    loop_batch_size = min(1000, total_num_faces)
    
    m_v = torch.zeros([max_m,max_m,max_m],dtype=torch.float32, requires_grad=True).to(device)
    m_s = torch.zeros([max_m,max_m,max_m],dtype=torch.float32, requires_grad=True).to(device)    
    
#     # This can be taken outside and made available as a global constant
#     f_ijk_s_p = np.zeros((max_m,max_m,max_m))
#     f_ijk_p_s_2 = np.zeros((max_m,max_m,max_m))
#     f_ijk_p_s_3 = np.zeros((max_m,max_m,max_m))
#     for i in range(max_m):
#                 for j in range(max_m):
#                     for k in range(max_m):
#                         f_ijk_s_p[i,j,k] = (factorial(i+j+k)/(factorial(i)*factorial(j)*factorial(k)))
#                         f_ijk_p_s_3[i,j,k] = ((factorial(i)*factorial(j)*factorial(k))/factorial(i+j+k+3))
#                         f_ijk_p_s_2[i,j,k] = 1/f_ijk_s_p[i,j,k]/(i+j+k+1)/(i+j+k+2)
#                         f_ijk_p_s_3[i,j,k] = f_ijk_p_s_2[i,j,k]/(i+j+k+3)
#     f_ijk_s_p = torch.tensor(f_ijk_s_p).float()
#     f_ijk_p_s_2 = torch.tensor(f_ijk_p_s_2).float()
#     f_ijk_p_s_3 = torch.tensor(f_ijk_p_s_3).float()
    
    
    
    for batch_num in range(ceil(total_num_faces/loop_batch_size)):
        start = batch_num*loop_batch_size
        end = min([(batch_num+1)*loop_batch_size, total_num_faces])
        num_faces = end-start
        
        # Extract coordinates
        ABC = v_matrix[f_matrix[start:end,:]]
        if is_watertigh:
            dets = torch.det(ABC).to(device)
        cross = torch.norm(torch.cross(ABC[:,0]-ABC[:,2],ABC[:,1]-ABC[:,2]),p=2,dim = 1).to(device)
        
        
        i = torch.arange(max_m)
        # Calculate C tensors
        C1_tensor = torch.multiply(torch.multiply(torch.pow(ABC[:,0][:,0].view(-1,1,1,1),i.view(1,-1,1,1)),torch.pow(ABC[:,0][:,1].view(-1,1,1,1),i.view(1,1,-1,1))),torch.pow(ABC[:,0][:,2].view(-1,1,1,1),i.view(1,1,1,-1)))*f_ijk_s_p
        C2_tensor = torch.multiply(torch.multiply(torch.pow(ABC[:,1][:,0].view(-1,1,1,1),i.view(1,-1,1,1)),torch.pow(ABC[:,1][:,1].view(-1,1,1,1),i.view(1,1,-1,1))),torch.pow(ABC[:,1][:,2].view(-1,1,1,1),i.view(1,1,1,-1)))*f_ijk_s_p
        C3_tensor = torch.multiply(torch.multiply(torch.pow(ABC[:,2][:,0].view(-1,1,1,1),i.view(1,-1,1,1)),torch.pow(ABC[:,2][:,1].view(-1,1,1,1),i.view(1,1,-1,1))),torch.pow(ABC[:,2][:,2].view(-1,1,1,1),i.view(1,1,1,-1)))*f_ijk_s_p
        
        
        # set up "contraction kernel"
        inds = torch.arange (max_m)
        kernel = ((inds.unsqueeze (0) + inds.unsqueeze (1)).unsqueeze (0) == inds.unsqueeze (-1).unsqueeze (-1)).float()

        D_tensor = torch.einsum ('eijk, ail, bjm, ckn, elmn -> eabc', C2_tensor, kernel, kernel, kernel, C3_tensor)
        
        S_tensor = torch.einsum ('eijk, ail, bjm, ckn, elmn -> eabc', C1_tensor, kernel, kernel, kernel, D_tensor) 
        
        m_v = m_v + (S_tensor * f_ijk_p_s_3 * dets.view(-1,1,1,1)).sum(0)
        m_s = m_s + (S_tensor * f_ijk_p_s_2 * cross.view(-1,1,1,1)).sum(0)
        
    return (m_v,m_s)



In [6]:
def M_ijk_Pozo_v1(f_matrix, v_matrix, max_m, is_watertigh, device):
    total_num_faces = len(f_matrix)
    
    loop_batch_size = min(1000, total_num_faces)
    
    m_v = torch.zeros([max_m,max_m,max_m],dtype=torch.float32, requires_grad=True).to(device)
    m_s = torch.zeros([max_m,max_m,max_m],dtype=torch.float32, requires_grad=True).to(device)    
    
#     # This can be taken outside and made available as a global constant
#     f_ijk_s_p = np.zeros((max_m,max_m,max_m))
#     f_ijk_p_s_2 = np.zeros((max_m,max_m,max_m))
#     f_ijk_p_s_3 = np.zeros((max_m,max_m,max_m))
#     for i in range(max_m):
#                 for j in range(max_m):
#                     for k in range(max_m):
#                         f_ijk_s_p[i,j,k] = (factorial(i+j+k)/(factorial(i)*factorial(j)*factorial(k)))
#                         f_ijk_p_s_3[i,j,k] = ((factorial(i)*factorial(j)*factorial(k))/factorial(i+j+k+3))
#                         f_ijk_p_s_2[i,j,k] = 1/f_ijk_s_p[i,j,k]/(i+j+k+1)/(i+j+k+2)
#                         f_ijk_p_s_3[i,j,k] = f_ijk_p_s_2[i,j,k]/(i+j+k+3)
#     f_ijk_s_p = torch.tensor(f_ijk_s_p).float()
#     f_ijk_p_s_2 = torch.tensor(f_ijk_p_s_2).float()
#     f_ijk_p_s_3 = torch.tensor(f_ijk_p_s_3).float()
    
    
    i = torch.arange(max_m)
    # set up "contraction kernel"
    kernel = ((i.unsqueeze (0) + i.unsqueeze (1)).unsqueeze (0) == i.unsqueeze (-1).unsqueeze (-1)).float()
        
    for batch_num in range(ceil(total_num_faces/loop_batch_size)):
        start = batch_num*loop_batch_size
        end = min([(batch_num+1)*loop_batch_size, total_num_faces])
        num_faces = end-start
        
        # Extract coordinates
        ABC = v_matrix[f_matrix[start:end,:]]
        if is_watertigh:
            dets = torch.det(ABC).to(device)
        cross = torch.norm(torch.cross(ABC[:,0]-ABC[:,2],ABC[:,1]-ABC[:,2]),p=2,dim = 1).to(device)
        
        S_tensor = torch.einsum ('eijk, ail, bjm, ckn, elmn -> eabc', torch.multiply(torch.multiply(torch.pow(ABC[:,0][:,0].view(-1,1,1,1),i.view(1,-1,1,1)),torch.pow(ABC[:,0][:,1].view(-1,1,1,1),i.view(1,1,-1,1))),torch.pow(ABC[:,0][:,2].view(-1,1,1,1),i.view(1,1,1,-1)))*f_ijk_s_p, kernel, kernel, kernel, torch.einsum ('eijk, ail, bjm, ckn, elmn -> eabc', torch.multiply(torch.multiply(torch.pow(ABC[:,1][:,0].view(-1,1,1,1),i.view(1,-1,1,1)),torch.pow(ABC[:,1][:,1].view(-1,1,1,1),i.view(1,1,-1,1))),torch.pow(ABC[:,1][:,2].view(-1,1,1,1),i.view(1,1,1,-1)))*f_ijk_s_p, kernel, kernel, kernel, torch.multiply(torch.multiply(torch.pow(ABC[:,2][:,0].view(-1,1,1,1),i.view(1,-1,1,1)),torch.pow(ABC[:,2][:,1].view(-1,1,1,1),i.view(1,1,-1,1))),torch.pow(ABC[:,2][:,2].view(-1,1,1,1),i.view(1,1,1,-1)))*f_ijk_s_p)) 
        
        if is_watertigh:
            m_v = m_v + (S_tensor * f_ijk_p_s_3 * dets.view(-1,1,1,1)).sum(0)
        m_s = m_s + (S_tensor * f_ijk_p_s_2 * cross.view(-1,1,1,1)).sum(0)
        
    return (m_v,m_s)



In [7]:
# Check watertight case
print('Watertight volumentric moments match:',torch.allclose(M_ijk_torch_v6(f_matrix,v_matrix,max_m,True,torch.device('cpu'))[0],M_ijk_Pozo_v1(f_matrix,v_matrix,max_m,True,torch.device('cpu'))[0]))
print('Watertight surface moments match:',torch.allclose(M_ijk_torch_v6(f_matrix,v_matrix,max_m,True,torch.device('cpu'))[1],M_ijk_Pozo_v1(f_matrix,v_matrix,max_m,True,torch.device('cpu'))[1]))


Watertight volumentric moments match: True
Watertight surface moments match: True


In [8]:
# Check Nonwatertight case
print('Nonwatertight volumentric moments Pozo (zeros):',torch.allclose(torch.zeros([max_m,max_m,max_m]),M_ijk_Pozo_v1(f_matrix,v_matrix,max_m,False,torch.device('cpu'))[0]))
print('Nonwatertight volumentric moments Koehl (zeros):',torch.allclose(torch.zeros([max_m,max_m,max_m]),M_ijk_torch_v6(f_matrix,v_matrix,max_m,False,torch.device('cpu'))[0]))
print('Nonwatertight surface moments match:',torch.allclose(M_ijk_torch_v6(f_matrix,v_matrix,max_m,False,torch.device('cpu'))[1],M_ijk_Pozo_v1(f_matrix,v_matrix,max_m,False,torch.device('cpu'))[1]))


Nonwatertight volumentric moments Pozo (zeros): True
Nonwatertight volumentric moments Koehl (zeros): True
Nonwatertight surface moments match: True


In [9]:
M_ijk_Pozo_v0(f_matrix,v_matrix,max_m,True,torch.device('cpu'))

(tensor([[[1.0000e+00, 5.0000e-01, 3.3333e-01,  ..., 7.6923e-02,
           7.1429e-02, 6.6667e-02],
          [5.0000e-01, 2.5000e-01, 1.6667e-01,  ..., 3.8462e-02,
           3.5714e-02, 3.3333e-02],
          [3.3333e-01, 1.6667e-01, 1.1111e-01,  ..., 2.5641e-02,
           2.3810e-02, 2.2222e-02],
          ...,
          [7.6923e-02, 3.8462e-02, 2.5641e-02,  ..., 5.9172e-03,
           5.4945e-03, 5.1282e-03],
          [7.1429e-02, 3.5714e-02, 2.3810e-02,  ..., 5.4945e-03,
           5.1020e-03, 4.7619e-03],
          [6.6667e-02, 3.3333e-02, 2.2222e-02,  ..., 5.1282e-03,
           4.7619e-03, 4.4444e-03]],
 
         [[5.0000e-01, 2.5000e-01, 1.6667e-01,  ..., 3.8462e-02,
           3.5714e-02, 3.3333e-02],
          [2.5000e-01, 1.2500e-01, 8.3333e-02,  ..., 1.9231e-02,
           1.7857e-02, 1.6667e-02],
          [1.6667e-01, 8.3333e-02, 5.5556e-02,  ..., 1.2821e-02,
           1.1905e-02, 1.1111e-02],
          ...,
          [3.8462e-02, 1.9231e-02, 1.2821e-02,  ..., 2.958

In [10]:
M_ijk_Pozo_v1(f_matrix,v_matrix,max_m,True,torch.device('cpu'))

(tensor([[[1.0000e+00, 5.0000e-01, 3.3333e-01,  ..., 7.6923e-02,
           7.1429e-02, 6.6667e-02],
          [5.0000e-01, 2.5000e-01, 1.6667e-01,  ..., 3.8462e-02,
           3.5714e-02, 3.3333e-02],
          [3.3333e-01, 1.6667e-01, 1.1111e-01,  ..., 2.5641e-02,
           2.3810e-02, 2.2222e-02],
          ...,
          [7.6923e-02, 3.8462e-02, 2.5641e-02,  ..., 5.9172e-03,
           5.4945e-03, 5.1282e-03],
          [7.1429e-02, 3.5714e-02, 2.3810e-02,  ..., 5.4945e-03,
           5.1020e-03, 4.7619e-03],
          [6.6667e-02, 3.3333e-02, 2.2222e-02,  ..., 5.1282e-03,
           4.7619e-03, 4.4444e-03]],
 
         [[5.0000e-01, 2.5000e-01, 1.6667e-01,  ..., 3.8462e-02,
           3.5714e-02, 3.3333e-02],
          [2.5000e-01, 1.2500e-01, 8.3333e-02,  ..., 1.9231e-02,
           1.7857e-02, 1.6667e-02],
          [1.6667e-01, 8.3333e-02, 5.5556e-02,  ..., 1.2821e-02,
           1.1905e-02, 1.1111e-02],
          ...,
          [3.8462e-02, 1.9231e-02, 1.2821e-02,  ..., 2.958

In [11]:
M_ijk_torch_v6(f_matrix,v_matrix,max_m,True,torch.device('cpu'))

(tensor([[[1.0000e+00, 5.0000e-01, 3.3333e-01,  ..., 7.6923e-02,
           7.1429e-02, 6.6667e-02],
          [5.0000e-01, 2.5000e-01, 1.6667e-01,  ..., 3.8462e-02,
           3.5714e-02, 3.3333e-02],
          [3.3333e-01, 1.6667e-01, 1.1111e-01,  ..., 2.5641e-02,
           2.3810e-02, 2.2222e-02],
          ...,
          [7.6923e-02, 3.8462e-02, 2.5641e-02,  ..., 5.9172e-03,
           5.4945e-03, 5.1282e-03],
          [7.1429e-02, 3.5714e-02, 2.3810e-02,  ..., 5.4945e-03,
           5.1020e-03, 4.7619e-03],
          [6.6667e-02, 3.3333e-02, 2.2222e-02,  ..., 5.1282e-03,
           4.7619e-03, 4.4444e-03]],
 
         [[5.0000e-01, 2.5000e-01, 1.6667e-01,  ..., 3.8462e-02,
           3.5714e-02, 3.3333e-02],
          [2.5000e-01, 1.2500e-01, 8.3333e-02,  ..., 1.9231e-02,
           1.7857e-02, 1.6667e-02],
          [1.6667e-01, 8.3333e-02, 5.5556e-02,  ..., 1.2821e-02,
           1.1905e-02, 1.1111e-02],
          ...,
          [3.8462e-02, 1.9231e-02, 1.2821e-02,  ..., 2.958

In [12]:
%timeit M_ijk_Pozo_v0(f_matrix,v_matrix,max_m,True,torch.device('cpu'))

486 ms ± 28.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [13]:
%timeit M_ijk_Pozo_v1(f_matrix,v_matrix,max_m,True,torch.device('cpu'))

473 ms ± 10.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [14]:
%timeit M_ijk_torch_v6(f_matrix,v_matrix,max_m,True,torch.device('cpu'))

681 ms ± 3.81 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
